In [ ]:
%cd /content/drive/MyDrive
!mkdir -p The_new_dir
%cd The_new_dir

!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

# Fix PyTorch loading issue
!sed -i 's/ckpt = torch.load(w, map_location=map_location)/ckpt = torch.load(w, map_location=map_location, weights_only=False)/g' models/experimental.py

from google.colab import files
import os
import glob
from IPython.display import display, Image
import shutil

# Upload file
uploaded = files.upload()
filename = list(uploaded.keys())[0]
file_path = filename

print(f"Processing file: {file_path}")

# Detect file type and run inference
image_formats = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
video_formats = ['.mp4', '.avi', '.mov', '.mkv']

file_ext = os.path.splitext(file_path)[1].lower()

if file_ext in image_formats:
    print("Running image detection...")
    !python detect.py --weights yolov7.pt --conf 0.5 --img-size 640 --source "{file_path}"

    # Display result
    output_dir = "runs/detect/"
    exp_folders = sorted(glob.glob(os.path.join(output_dir, "exp*")))

    if exp_folders:
        latest_exp = exp_folders[-1]
        output_path = os.path.join(latest_exp, filename)

        if os.path.exists(output_path):
            display(Image(output_path, width=600))
            print("Image detection completed!")

elif file_ext in video_formats:
    print("Running video detection...")
    !python detect.py --weights yolov7.pt --conf 0.5 --img-size 640 --source "{file_path}"

    # Display result
    output_dir = "runs/detect/"
    exp_folders = sorted(glob.glob(os.path.join(output_dir, "exp*")))

    if exp_folders:
        latest_exp = exp_folders[-1]
        output_path = os.path.join(latest_exp, filename)

        if os.path.exists(output_path):
            # Auto-download the output video
            print("Auto-downloading processed video...")
            from google.colab import files as colab_files
            colab_files.download(output_path)
            print("Video downloaded! Check your downloads folder.")

            # Show download confirmation with file info
            file_size = os.path.getsize(output_path) / (1024*1024)  # Size in MB
            print("Video Processing Complete!")
            print(f"File: {filename}")
            print(f"Size: {file_size:.2f} MB")
            print("Status: Auto-downloaded to your computer")
            print("The processed video has been automatically downloaded. Check your browser's downloads.")
        else:
            # Try to find any video file in the directory
            video_files = glob.glob(os.path.join(latest_exp, "*"))
            for vid_file in video_files:
                if any(vid_file.lower().endswith(ext) for ext in video_formats):
                    print(f"Found output video: {vid_file}")
                    print("Auto-downloading processed video...")
                    from google.colab import files as colab_files
                    colab_files.download(vid_file)
                    print("Video downloaded! Check your downloads folder.")
                    break

print("Detection completed!")